# T4 - Filtering & time series
Before we look at the full (multivariate) Kalman filter,
let's get more familiar with time-dependent (temporal/sequential) problems.
$
% Loading TeX (MathJax)... Please wait
%
\newcommand{\Reals}{\mathbb{R}}
\newcommand{\Expect}[0]{\mathbb{E}}
\newcommand{\NormDist}{\mathcal{N}}
%
\newcommand{\DynMod}[0]{\mathscr{M}}
\newcommand{\ObsMod}[0]{\mathscr{H}}
%
\newcommand{\mat}[1]{{\mathbf{{#1}}}}
%\newcommand{\mat}[1]{{\pmb{\mathsf{#1}}}}
\newcommand{\bvec}[1]{{\mathbf{#1}}}
%
\newcommand{\trsign}{{\mathsf{T}}}
\newcommand{\tr}{^{\trsign}}
\newcommand{\tn}[1]{#1}
\newcommand{\ceq}[0]{\mathrel{≔}}
%
\newcommand{\I}[0]{\mat{I}}
\newcommand{\K}[0]{\mat{K}}
\newcommand{\bP}[0]{\mat{P}}
\newcommand{\bH}[0]{\mat{H}}
\newcommand{\bF}[0]{\mat{F}}
\newcommand{\R}[0]{\mat{R}}
\newcommand{\Q}[0]{\mat{Q}}
\newcommand{\B}[0]{\mat{B}}
\newcommand{\C}[0]{\mat{C}}
\newcommand{\Ri}[0]{\R^{-1}}
\newcommand{\Bi}[0]{\B^{-1}}
\newcommand{\X}[0]{\mat{X}}
\newcommand{\A}[0]{\mat{A}}
\newcommand{\Y}[0]{\mat{Y}}
\newcommand{\E}[0]{\mat{E}}
\newcommand{\U}[0]{\mat{U}}
\newcommand{\V}[0]{\mat{V}}
%
\newcommand{\x}[0]{\bvec{x}}
\newcommand{\y}[0]{\bvec{y}}
\newcommand{\z}[0]{\bvec{z}}
\newcommand{\q}[0]{\bvec{q}}
\newcommand{\br}[0]{\bvec{r}}
\newcommand{\bb}[0]{\bvec{b}}
%
\newcommand{\bx}[0]{\bvec{\bar{x}}}
\newcommand{\by}[0]{\bvec{\bar{y}}}
\newcommand{\barB}[0]{\mat{\bar{B}}}
\newcommand{\barP}[0]{\mat{\bar{P}}}
\newcommand{\barC}[0]{\mat{\bar{C}}}
\newcommand{\barK}[0]{\mat{\bar{K}}}
%
\newcommand{\D}[0]{\mat{D}}
\newcommand{\Dobs}[0]{\mat{D}_{\text{obs}}}
\newcommand{\Dmod}[0]{\mat{D}_{\text{obs}}}
%
\newcommand{\ones}[0]{\bvec{1}}
\newcommand{\AN}[0]{\big( \I_N - \ones \ones\tr / N \big)}
%
% END OF MACRO DEF
$

In [ ]:
import resources.workspace as ws
%matplotlib inline
import numpy as np
import numpy.random as rnd
import matplotlib.pyplot as plt
plt.ion();

Scalar, auto-regressive stochastic process of order 1, i.e. AR(1).

In [ ]:
# Use H=1 so that it makes sense to plot data on same axes as state.
H = 1

xa = 0   # initial estimate mean
Pa = 10  # initial estimate variance

def simulate(K, xa, Pa, M, H, Q, R):
    """Simulate synthetic truth (x) and observations (y)."""
    x = xa + np.sqrt(Pa)*rnd.randn()        # Draw initial condition
    truths = np.zeros(K)                    # Allocate
    obsrvs = np.zeros(K)                    # Allocate
    for k in range(K):                      # Loop in time
        x = M * x + np.sqrt(Q)*rnd.randn()  # Dynamics
        y = H * x + np.sqrt(R)*rnd.randn()  # Measurement
        truths[k] = x                       # Assign
        obsrvs[k] = y                       # Assign
    return truths, obsrvs

If you find that there are not enough sliders to play around with,
feel free to alter the code to suit your needs
(for example, you can comment out the line that is plotting the observations,
or the line plotting the uncertainty CI).

In [ ]:
@ws.interact(seed=(1, 12), M=(0, 1.03, .01), K=(0, 100),
             logR=(-9, 9), logR_bias=(-9, 9),
             logQ=(-9, 9), logQ_bias=(-9, 9))
def plot_experiment(seed, K, M=0.97, logR=1, logQ=1, analyses_only=False, logR_bias=0, logQ_bias=0):
    R, Q, Q_bias, R_bias = 4.0**np.array([logR, logQ, logQ_bias, logR_bias])

    rnd.seed(seed)
    truths, obsrvs = simulate(K, xa, Pa, M, H, Q, R)

    plt.figure(figsize=(9, 6))
    kk = 1 + np.arange(K)
    plt.plot(kk, truths, 'k' , label='True state ($x$)')
    plt.plot(kk, obsrvs, 'g*', label='Noisy obs ($y$)', ms=9)

    try:
        estimates, variances = KF(K, xa, Pa, M, H, Q*Q_bias, R*R_bias, obsrvs)
        # +/- 1-sigma (std.dev.) credible intervals (CI):
        upper = estimates + np.sqrt(variances)
        lower = estimates - np.sqrt(variances)
        if analyses_only:
            plt.plot(kk, estimates[:, 1], label='Kalman$^a$ ± 1$\sigma$')
            plt.fill_between(kk, lower[:, 1], upper[:, 1], alpha=.2)
        else:
            kk = kk.repeat(2)
            plt.plot(kk, estimates.flatten(), label='Kalman ± 1$\sigma$')
            plt.fill_between(kk, lower.flatten(), upper.flatten(), alpha=.2)
    except NameError:
        pass

    plt.xlabel('Time index (k)')
    plt.legend(loc='upper left')
    plt.axhline(0, c='k', lw=1, ls='--')
    plt.grid()
    plt.show()

**Exc 4.4:** Answer the following.
- What does `seed` control?
- Explain what happens when `M=0`. Also consider $Q \rightarrow 0$.  
  Can you give a name to this `truth` process,
  i.e. a link to the relevant Wikipedia page?  
  What about when `M=1`?  
  Describe the general nature of the process as `M` changes from 0 to 1.  
  What about when `M>1`?  
- What happens when $R \rightarrow 0$ ?
- What happens when $R \rightarrow \infty$ ?

In [ ]:
# ws.show_answer('AR1')

#### Exc 4.6 -- Implementing the scalar Kalman filter (KF)
Below is a very rudimentary filter, essentially just doing "persistance" forecasts, and setting the analysis estimates to the value of the observations (*which is only generally a possibility in this linear, scalar case*). Run its cell to define it, and then re-run the above interactive animation cell.
- Implement the KF properly by replace the forecast and analysis steps below. *Re-run the cell.*
- Also try implementing (replacing) the analysis step by the "gain form" of the KF.

In [ ]:
def KF(K, xa, Pa, M, H, Q, R, obsrvs):
    """Kalman filter."""
    ############################
    # TEMPORARY IMPLEMENTATION #
    ############################
    estimates = np.zeros((K, 2))
    variances = np.zeros((K, 2))
    for k in range(K):
        # Forecast step
        xf = xa
        Pf = Pa
        # Analysis update step
        Pa = R / H**2
        xa = obsrvs[k] / H
        # Assign
        estimates[k] = xf, xa
        variances[k] = Pf, Pa
    return estimates, variances

In [ ]:
def KF(K, xa, Pa, M, H, Q, R, obsrvs):
    estimates = np.zeros((K, 2))
    variances = np.zeros((K, 2))
    for k in range(K):
        # Forecast step
        xf = M * xa
        Pf = M**2 * Pa + Q
        # Analysis update step
        Pa = 1 / (1/Pf + H**2/R)
        xa = Pa * (xf/Pf + H*obsrvs[k]/R)
        # Assign
        estimates[k] = xf, xa
        variances[k] = Pf, Pa
    return estimates, variances

#### Exc
- Set `logQ` to its minimum, and `M=1`.  
  We established in Exc 4.4 that the true states are now constant in time (but unknown).  
  How does the KF fare in estimating it?  
  Does its uncertainty variance ever reach 0?
- What is the KF uncertainty variance in the case of `M=0`?

In [ ]:
# ws.show_answer('KF behaviour')

#### Exc 3.14 -- temporal asymptotic convergence
In general, $\DynMod$, $\ObsMod$, $Q$, and $R$ depend on time, $k$
(often to parameterize exogenous/outside factors/forces/conditions),
and there are no limit values that the KF parameters converge to.
But, let $Q=0$ in eqn (Dyn) so that $x_{k+1} = \DynMod x_k$ for some *constant* $\DynMod$.
Show that sequence of $P_k^a$ converges to
- (a) 0 if $\DynMod < 1$.  
  *Hint: Merge the analysis and forecast equations.*
- (b) 0 if $\DynMod = 1$.
- (c) $R (1-1/\DynMod^2)$ if $\DynMod > 1$.  
  *Hint: Look for the fixed point.*

In [ ]:
# ws.show_answer('Asymptotic Riccati')

#### Exc 4.11 -- impact of biases
Re-run the above interative animation to set the default control values. Answer the following

- `logR_bias`/`logQ_bias` control the (multiplicative) bias in $R$/$Q$ that is fed to the KF.
  What happens when the KF "thinks" the measurement/dynamical error
  is (much) smaller than it actually is?
  What about larger?
- Re-run the animation to get default values.
  Set `logQ` to 0, which will make the following behaviour easier to describe.
  In the code, add 20 to the initial `xa` **given to the KF**.
  How long does it take for it to recover from this initial bias?
- Multiply `Pa` **given to the KF** by 0.01. What about now?

In [ ]:
# ws.show_answer('KF with bias')